In [1]:
import requests
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Setting up

In [2]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
json = response.json()

In [3]:
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

In [4]:
slim_elements_df = elements_df[['second_name', 'team', 'element_type', 'selected_by_percent',
                                'now_cost', 'minutes', 'transfers_in', 'value_season', 'form', 'total_points']]

slim_elements_df['position'] = slim_elements_df.element_type.map(
    elements_types_df.set_index('id').singular_name)

slim_elements_df['team'] = slim_elements_df.team.map(
    teams_df.set_index('id').name)

slim_elements_df['value'] = slim_elements_df.value_season.astype(float)

# removing everyone with 0 value
slim_elements_df = slim_elements_df.loc[slim_elements_df.value > 0]


# Analyzing

In [5]:
# Sorting by value
slim_elements_df.sort_values('value',ascending=False).head(10)

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,form,total_points,position,value
679,Coady,Wolves,2,18.1,49,2686,2028774,25.1,3.9,123,Defender,25.1
700,Malheiro de Sá,Wolves,1,12.4,53,2700,1663506,24.7,3.3,131,Goalkeeper,24.7
406,Cancelo,Man City,2,36.5,69,2417,4962796,23.5,5.2,162,Defender,23.5
364,Matip,Liverpool,2,3.8,49,2070,861085,23.1,6.6,113,Defender,23.1
371,Ramses Becker,Liverpool,1,8.0,60,2430,817728,23.0,7.0,138,Goalkeeper,23.0
32,Ramsdale,Arsenal,1,21.4,51,2160,2784949,22.9,2.7,117,Goalkeeper,22.9
658,Bowen,West Ham,3,12.2,68,2361,3899127,22.5,1.0,153,Midfielder,22.5
56,Cash,Aston Villa,2,9.7,53,2477,1312387,22.3,8.2,118,Defender,22.3
689,Kilman,Wolves,2,4.2,46,2508,554712,22.2,2.0,102,Defender,22.2
432,de Gea,Man Utd,1,10.5,51,2610,1834508,22.2,2.8,113,Goalkeeper,22.2


In [6]:
# Avg value per position
position_pivot = slim_elements_df.pivot_table(index='position',values='value',aggfunc=np.mean).reset_index()

position_pivot.sort_values('value',ascending=False)

,position,value
2,Goalkeeper,10.638462
0,Defender,9.315254
3,Midfielder,7.778414
1,Forward,6.538889


In [7]:
team_pivot = slim_elements_df.pivot_table(index='team',values='value',aggfunc=np.mean).reset_index()

team_pivot.sort_values('value',ascending=False)

,team,value
19,Wolves,10.729167
5,Chelsea,10.492000
11,Man City,10.024000
6,Crystal Palace,9.870833
10,Liverpool,9.788889
0,Arsenal,9.504000
16,Spurs,9.295833
18,West Ham,9.292000
3,Brighton,8.512500
4,Burnley,8.509524


In [8]:
# Goalkeepers
goal_df = slim_elements_df.loc[slim_elements_df.position == 'Goalkeeper']

goal_df.sort_values('value',ascending=False).head(5)

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,form,total_points,position,value
700,Malheiro de Sá,Wolves,1,12.4,53,2700,1663506,24.7,3.3,131,Goalkeeper,24.7
371,Ramses Becker,Liverpool,1,8.0,60,2430,817728,23.0,7.0,138,Goalkeeper,23.0
32,Ramsdale,Arsenal,1,21.4,51,2160,2784949,22.9,2.7,117,Goalkeeper,22.9
432,de Gea,Man Utd,1,10.5,51,2610,1834508,22.2,2.8,113,Goalkeeper,22.2
563,Lloris,Spurs,1,7.9,54,2520,1191018,20.7,3.3,112,Goalkeeper,20.7


In [9]:
# Defenders
def_df = slim_elements_df.loc[slim_elements_df.position == 'Defender']

def_df.sort_values('value',ascending=False).head(10)

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,form,total_points,position,value
679,Coady,Wolves,2,18.1,49,2686,2028774,25.1,3.9,123,Defender,25.1
406,Cancelo,Man City,2,36.5,69,2417,4962796,23.5,5.2,162,Defender,23.5
364,Matip,Liverpool,2,3.8,49,2070,861085,23.1,6.6,113,Defender,23.1
56,Cash,Aston Villa,2,9.7,53,2477,1312387,22.3,8.2,118,Defender,22.3
689,Kilman,Wolves,2,4.2,46,2508,554712,22.2,2.0,102,Defender,22.2
369,van Dijk,Liverpool,2,19.3,68,2430,1944137,22.1,6.8,150,Defender,22.1
377,Alexander-Arnold,Liverpool,2,44.0,85,2313,4176136,22.0,6.2,187,Defender,22.0
25,White,Arsenal,2,13.8,45,2250,1476580,21.3,2.2,96,Defender,21.3
409,Laporte,Man City,2,8.9,58,2082,1633785,21.2,4.0,123,Defender,21.2
681,Saïss,Wolves,2,5.2,51,2468,827830,20.8,2.6,106,Defender,20.8


In [10]:
# Midfielders
mid_df = slim_elements_df.loc[slim_elements_df.position == 'Midfielder']

mid_df.sort_values('value',ascending=False).head(10)

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,form,total_points,position,value
658,Bowen,West Ham,3,12.2,68,2361,3899127,22.5,1.0,153,Midfielder,22.5
220,Gallagher,Crystal Palace,3,12.5,61,2199,4426484,20.2,3.6,123,Midfielder,20.2
63,Ramsey,Aston Villa,3,9.8,48,1860,1361788,19.6,3.5,94,Midfielder,19.6
19,Saka,Arsenal,3,31.2,67,2067,4332822,19.0,4.7,127,Midfielder,19.0
411,Veiga de Carvalho e Silva,Man City,3,15.9,71,2407,4113166,18.5,4.2,131,Midfielder,18.5
18,Smith Rowe,Arsenal,3,11.0,57,1408,3763063,18.4,2.2,105,Midfielder,18.4
362,Henderson,Liverpool,3,1.8,50,2000,462673,18.0,3.8,90,Midfielder,18.0
657,Benrahma,West Ham,3,11.6,59,1701,4162407,17.8,2.8,105,Midfielder,17.8
573,Højbjerg,Spurs,3,3.0,49,2296,629807,17.3,2.4,85,Midfielder,17.3
31,Ødegaard,Arsenal,3,5.9,55,1797,1299030,17.3,2.8,95,Midfielder,17.3


In [11]:
# Forwards
fwd_df = slim_elements_df.loc[slim_elements_df.position == 'Forward']

fwd_df.sort_values('value',ascending=False).head(10)

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,form,total_points,position,value
626,Dennis,Watford,4,30.1,60,2174,6058308,19.2,3.2,115,Forward,19.2
499,Pukki,Norwich,4,5.8,58,2445,1239910,17.9,4.6,104,Forward,17.9
84,Toney,Brentford,4,14.9,67,2098,3023427,16.0,6.0,107,Forward,16.0
651,Antonio,West Ham,4,29.6,75,2431,7324412,15.1,1.8,113,Forward,15.1
552,Broja,Southampton,4,13.9,55,1510,2224088,14.7,1.8,81,Forward,14.7
128,Maupay,Brighton,4,7.4,65,2042,1927059,14.5,1.4,94,Forward,14.5
629,King,Watford,4,4.2,57,1966,1835944,13.9,1.2,79,Forward,13.9
481,Saint-Maximin,Newcastle,4,11.5,68,2042,3423815,13.8,0.5,94,Forward,13.8
542,Adams,Southampton,4,5.7,69,1645,970625,13.3,4.0,92,Forward,13.3
249,Edouard,Crystal Palace,4,3.7,62,1263,1599415,13.1,0.2,81,Forward,13.1
